In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls 'xxxxxx/en-ach/tokenizer'

source.spm		 target.spm		vocab.json
special_tokens_map.json  tokenizer_config.json


In [1]:
!pip install --no-cache-dir transformers sentencepiece

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 1.1MB 50.9MB/s 
     |████████████████████████████████| 890kB 64.5MB/s 
     |████████████████████████████████| 2.9MB 35.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b7be627da02ddc8b6e2fa2b67402cd6bcb605b6608af28c4d0736fb331a581b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-qsooz_f8/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import numpy as np
import pandas as pd

##Data Collection

In [ ]:
!pip install opustools-pkg

In [ ]:
! opus_read -d JW300 -s ach -t en -wm moses -w jw300.ach jw300.en -q


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/ach-en.xml.gz not found. The following files are available for downloading:

 724 KB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ach-en.xml.gz
   8 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ach.zip
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip

 272 MB Total size
./JW300_latest_xml_ach-en.xml.gz ... 100% of 724 KB
./JW300_latest_xml_ach.zip ... 100% of 8 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB


In [ ]:
!opus_get --directory JW300 --source ach --target en

In [ ]:
source_language="en"
target_language="ach"

In [ ]:
# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
            source.append(line.strip())
                     
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
            target.append(line.strip())
       
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(5)

,source_sentence,target_sentence
0,Table of Contents,Lok ma Tye i Iye
1,"March 1 , 2011","Marci 1 , 2011"
2,The “ Good News of the Kingdom ” ​ — What Is It ?,“ Kwena Maber me Ker ” Obedo Gin Ango ?
3,FROM OUR COVER,PWONY MA KWAKO LOK MA I POK NGEYE
4,3 A Prophecy of Enormous Importance,3 Lok Mo ma Pire Tek pa Lanebi


##Data Preprocessing and Preparation

In [ ]:
seed=23

In [ ]:
len(df)

81969

In [ ]:
df.replace('', np.nan, inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df=df.applymap(str)

In [ ]:
len(df)

79310

In [ ]:
#removing rows with cells that contain only non-alphabetic characters
df=df[df['source_sentence'].str.contains('[A-Za-z]')]
df=df[df['target_sentence'].str.contains('[A-Za-z]')]
len(df)

76402

In [ ]:
# drop duplicate translations
df_pp=df.copy()
df_pp = df_pp.drop_duplicates()

# drop conflicting translations
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
df_pp.reset_index(drop=True, inplace=True)

In [ ]:
df_pp.head()

,source_sentence,target_sentence
0,As ‘ Jehovah’s people and the flock of his pas...,"( Nwo . 32 : 7 ) Macalo ‘ lwak pa Jehovah , ma..."
1,"How will he be “ cast out , ” and how has he b...","I yo ango ma “ gibibole woko , ” dok “ gingoll..."
2,"By faith , however , you too can know Jehovah ...","Ento pi niyeni , in bene itwero ngeyo Jehovah ..."
3,"No doubt , school counselors sincerely believe...",Wangeyo gire ni jo ma gimini tam ki i cukul pe...
4,Even those who get baptized when they are adul...,Kadi wa jo ma ginongo batija ma dong guteggi p...


In [ ]:
size=len(df_pp)
size

73092

In [ ]:
# Split between train/dev/test- ratio 7:2:1 for then saves them as separate files
import csv

lc=True
# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc: 
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

train_size=int(size*0.7)
train=df_pp.iloc[:train_size, 0:2]

dev_size=int(size*0.2)
end=train_size+dev_size
dev=df_pp.iloc[train_size:end, 0:2]


test_size=int(size*0.1)
test=df_pp.iloc[end:, 0:2]


with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in train.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

df_pp.to_csv(source_language +"-"+ target_language, header=False, index=False) 


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.ach <==
( nwo . 32 : 7 ) macalo ‘ lwak pa jehovah , ma wan romine ma en kwayowa - ni , ’ wan ducu wapako lubanga kun watito pi tic matego ma en otimo . ( jab .
i yo ango ma “ gibibole woko , ” dok “ gingolle kop ” ?
ento pi niyeni , in bene itwero ngeyo jehovah maber adada , ma bedo calo iwacci itye ka nenne .
wangeyo gire ni jo ma gimini tam ki i cukul pe gitye ki cwiny marac ka gimini tam ni myero imedde ki kwan i rwom mamalo wek inong dog tic maber .
kadi wa jo ma ginongo batija ma dong guteggi pud giwok ki i atematema pi niyegi i yo ma pe gubyeko wa ki acel .
pingo pe omyero wawek latin kwan me baibul okwan lagamme akwana ki i buk ?
tika ibalo kero ki cawa ni i kom jami ma konygi pe ?
nyo bene wan kikomwa watwero nongo ni watye i kwo matek nyo pi lok me ngolo kop alenga .
wek wayer lok mupore , mitte ni myero watute dok wati ki wiwa .
pingo omyero watute me lubo lanen pa kabaka solomon ?

==> train.en <==
as ‘ jehovah’s people and the flock of his pasturage , ’ all of us 

In [ ]:
print((train_size,dev_size, test_size))

(51164, 14618, 7309)


# Building the Model

In [ ]:
device = 'cpu'
if torch.cuda.is_available(): #check if GPU device is available
    device = 'cuda' # assign the gpu to the device

In [7]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-luo", use_fast=False)
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-luo")
model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-en-luo")


In [ ]:
model = model.to(device) # bind the model to the GPU device

##Tokenizing

In [ ]:
src_texts = [ "Hello World"]

In [ ]:
tokens=tokenizer.prepare_seq2seq_batch(src_texts,)

In [ ]:
tokens

{'input_ids': [[44, 23418, 1006, 0]], 'attention_mask': [[1, 1, 1, 1]]}

In [ ]:
for k in tokens.keys():
  tokens[k]=torch.LongTensor(tokens[k])

In [ ]:
type(tokens)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
tokens.to(device)

{'input_ids': tensor([[   44, 23418,  1006,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}

In [ ]:
translated = model.generate(**tokens)

In [ ]:
tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
tgt_text

['MANA Kaka Piny Biro Mulo Ngimani']

In [ ]:
train_tokens=tokenizer.prepare_seq2seq_batch(list(train["source_sentence"]), tgt_texts=list(train["target_sentence"]), padding=True,truncation=True, return_tensors="pt" )
train_tokens.to(device)

{'input_ids': tensor([[   54,    21,  4774,  ..., 52235, 52235, 52235],
        [  166,    52,    53,  ..., 52235, 52235, 52235],
        [   79,   264,    44,  ..., 52235, 52235, 52235],
        ...,
        [   36,   267,    44,  ..., 52235, 52235, 52235],
        [  166,   227,     5,  ..., 52235, 52235, 52235],
        [  132,  3509,  4332,  ..., 52235, 52235, 52235]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([[   39, 20011,    44,  ..., 52235, 52235, 52235],
        [ 1474,   123,   118,  ..., 52235, 52235, 52235],
        [   32,  1925,   933,  ..., 52235, 52235, 52235],
        ...,
        [  152,  6919, 32572,  ..., 52235, 52235, 52235],
        [  933,    44, 23518,  ..., 52235, 52235, 52235],
        [ 2550,   243, 19909,  ..., 522

##Testing the model before fine-tuning

In [ ]:
def test_model(model,tokenizer, test_df):

  test_predictions=[]

  for row in test_df["source_sentence"]:
    tokens=tokenizer.prepare_seq2seq_batch([row])
    tokens.to(device)
    translated = model.generate(**tokens)
    tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    test_predictions.append(tgt_text[0])

  reference_preds=pd.DataFrame({"Predictions": test_predictions, "Reference": test["target_sentence"]})

  return pred_ref_df


In [ ]:
def gen_bleu_score(pred_ref_df):
  from nltk.translate.bleu_score import corpus_bleu
  ref_tokens=[]
  pred_tokens=[]

  for row in pred_ref_df["Reference"]:
    ref_tokens.append(row.split())
  for row in pred_ref_df["Predictions"]:
   pred_tokens.append(row.split())

   return corpus_bleu(ref_tokens, pred_tokens)


In [ ]:
before_pred_df=test_model(model, tokenizer, test)

In [ ]:
dev.reset_index(inplace=True, drop=True)

create the custom dataset that will be sued for dataloading

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, data, maxlen, tokenizer, with_labels=True,):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = tokenizer  

        # self.tokenized_data = self.tokenizer

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # print(type(self.data))
        # print("index:", index)

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = [self.data.loc[index, "source_sentence"]]
        sent2 = [self.data.loc[index, "target_sentence"]]

        # print("Sent1:", len(sent1))
        # print(sent1)
        # print("Sent2:", len(sent2))
        # print(sent2)

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer.prepare_seq2seq_batch(sent1, sent2, 
                                      # max_length=tokenizer.max_len,
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      return_tensors='pt')  # Return torch.Tensor objects

         
        # for key in encoded_pair.keys():
          # print("encoded pair : %s : %s"%(key, str(encoded_pair[key].size())))
          # print(encoded_pair[key])
          # print(tokenizer.convert_ids_to_tokens(encoded_pair[key].squeeze(0)))

        encoded_pair['input_ids'] = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        encoded_pair['attention_mask'] = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values

        # print("ids:", input_ids.size())
        # print(attn_masks.size())

        if self.with_labels:  # True if the dataset has labels
            encoded_pair['labels'] = encoded_pair['labels'].squeeze(0)
            # return input_ids, attn_masks, label   -- This is what I saw in an example. It resulted to an erroe
            return encoded_pair
            # return sent1, sent2
        else:
            # return sent1, sent2
            return input_ids, attn_masks

In [ ]:
train_dataset= CustomDataset(train, 1024, tokenizer, with_labels=True,)
eval_dataset= CustomDataset(dev, 1024, tokenizer, with_labels=True,)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #tokenizer=tokenizer
)

In [ ]:
trainer.train()  

Step,Training Loss
500,2.427850
1000,0.247516
1500,0.208337
2000,0.190387
2500,0.184724
3000,0.174153
3500,0.161937
4000,0.156642
4500,0.154239
5000,0.149986


TrainOutput(global_step=25582, training_loss=0.17011348998590403)

In [ ]:
#If you have enough memory - you can run the cell below otherwise run the next cell

test_tokens=tokenizer.prepare_seq2seq_batch(list(test["source_sentence"]),padding=True,truncation=True, return_tensors="pt" )
test_tokens.to(device)

{'input_ids': tensor([[  276,  3088,    12,  ..., 52235, 52235, 52235],
        [   70,    44,     3,  ..., 52235, 52235, 52235],
        [ 7473,   469,    44,  ..., 52235, 52235, 52235],
        ...,
        [   86,    29,     5,  ..., 52235, 52235, 52235],
        [ 3834,    44,     3,  ..., 52235, 52235, 52235],
        [   41,   663,  1363,  ..., 52235, 52235, 52235]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [ ]:
trainer.save_model("model/")

In [ ]:
test_predictions = [tokenizer.decode(t, skip_special_tokens=True) for t in model.generate(**test_tokens)]

In [ ]:
test_predictions=[]

for row in test["source_sentence"]:
    tokens=tokenizer.prepare_seq2seq_batch([row],padding=True,truncation=True, return_tensors="pt" )
    tokens.to(device)
    translated = model.generate(**tokens)
    tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    test_predictions.append(tgt_text[0])

In [ ]:
pred_len=len(test_predictions)

In [ ]:
pred_len

7310

In [ ]:
test_predictions

['en owaco ni :  ka ce jehovah pe mara, ci tirane onongo pe tye i yo me cwiny. ',
 '1, 2. ( a ) kuc ango ma watye kwede i kare - ni?',
 'i kare - ni bene, omyero wange ni mic pa kric, ma twero konyowa me timo bal.',
 'mar ma gitye kwede pi jehovah kacel ki niye matek i kom cikke ni en bijwayo jami ducu maraco ducu dok bimiyo mot bot luticce ma lugen me kwo ma pe tum i lobo manyen.  kwan jabuli 37 : 5, 7, 9, 29, 29, 29.',
 'i yo ango ma luelda gitye ka miyo yomcwiny bot luye luwo?',
 'dano pe  pe ye gin ma cwiny pa lubanga cwiny maleng, pien gitye ka tim maraco i kome, dok pe twero ngeyogi. ',
 'mitte ni wabed ki miti me ngeyo yubbe kacel ki yubbe. ',
 'ento i nge kare mo, en okwanyo yo acel keken i nge kare mukene.',
 'jo me lobo - ni gitye ka tito kwena i yo me cwiny, dok bene tye ka pwonyo jami mapol.',
 'tekcwiny i kine mukene nyuto gin ma pe wawaco.',
 'go kong lok man : myero ibed ka wot i kabedo mo.',
 'i kare ma en tye ka wot ma kiloko i cok, en ocako c i dye tim meno.',
 'lakwe

In [ ]:
ref=test.iloc[:pred_len, 1].values.tolist()

In [ ]:
reference_preds=pd.DataFrame({"Predictions": test_predictions, "Reference":ref})

In [ ]:
reference_preds.head(20)

,Predictions,Reference
0,"en owaco ni : ka ce jehovah pe mara, ci tiran...","en owaco ni : “ ka ni jehovah onongo pe mara ,..."
1,"1, 2. ( a ) kuc ango ma watye kwede i kare - ni?","1 , 2 . ( a ) watwero bedo ki kuc i kare - ni ..."
2,"i kare - ni bene, omyero wange ni mic pa kric,...","i kare - ni bene , omyero wange ni wamito gink..."
3,mar ma gitye kwede pi jehovah kacel ki niye ma...,margi i kom jehovah ki niyegi matek i kom cikk...
4,i yo ango ma luelda gitye ka miyo yomcwiny bot...,i yo ango ma luelda gimedo yomcwiny pa luye lu...
5,dano pe pe ye gin ma cwiny pa lubanga cwiny m...,dano ma pe ki cwiny pa lubanga nyo ma pe lubo ...
6,mitte ni wabed ki miti me ngeyo yubbe kacel ki...,ento wamito mere niang pulanne kacel ki yubbe . ”
7,"ento i nge kare mo, en okwanyo yo acel keken i...","ento lacen , en ocako lubo motmot yo marac ."
8,jo me lobo - ni gitye ka tito kwena i yo me cw...,lobo - ni cuko cwiny dano me kwan i rwom mamalo .
9,tekcwiny i kine mukene nyuto gin ma pe wawaco.,i kine mukene tekcwinywa nen i tic ma watimo e...


In [ ]:
reference_preds.to_csv("en-ach predictions.csv", header= True)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
ref_tokens=[]
pred_tokens=[]

In [ ]:
for row in ref:
  ref_tokens.append(row.split())

In [ ]:
for row in test_predictions:
   pred_tokens.append(row.split())

In [ ]:
len(pred_tokens)

7310

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

In [ ]:
corpus_bleu(ref_tokens, pred_tokens, weights=(1, 0))

0.06987352812908854

In [ ]:
!cp -R "/content/results/checkpoint-25500" 'xxxxxxxx' #moving to gdrive for permanent saving

In [ ]:
!cp -R tokenizer 'xxxxx' #moving to gdrive for permanent saving

In [ ]:
!cp -R model 'xxxxxx'

In [ ]:
tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 PosixPath('tokenizer/source_spm'),
 PosixPath('tokenizer/target_spm'),
 PosixPath('tokenizer/vocab'),
 PosixPath('tokenizer/tokenizer_config_file'),
 './tokenizer/added_tokens.json')